In [14]:
from tqdm import tqdm

In [15]:
import pandas as pd
parameter_frame = pd.read_excel(r'../../inputs/parameters.xlsx').set_index(['category','parameter'])
scenario_list = list(parameter_frame.columns)

In [16]:
pool = {}

In [17]:
scenario_list

['base', 'project']

In [18]:
df_dict = pd.read_excel('../../model/base/stacks.xlsx', sheet_name=None)
for key in df_dict.keys():
    pool[key] = []
for scenario in tqdm(scenario_list):
    try:
        df_dict = pd.read_excel(r'../../model/' + scenario + '/stacks.xlsx', sheet_name=None)
        for key, value in df_dict.items():
            value['scenario'] = scenario
            col = [c for c in value.columns if 'scenario' not in c]
            col.insert(-1, 'scenario')
            value = value[col]
            pool[key].append(value)
    except FileNotFoundError: # the stacks have not been produced yet:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.16it/s]


In [19]:
stacks = {k: pd.concat(v) for k, v in pool.items()}
stacks['parameters'] = parameter_frame.reset_index()
with pd.ExcelWriter(r'../../outputs/'+ 'stacks.xlsx') as writer:  # doctest: +SKIP
    for name, stack in stacks.items():
        stack.to_excel(writer, sheet_name=name, index=False)